In [7]:
#Description:
#   Initial and simple analysis of FBI Crime Data prior to 7/11 class
#   victim-data-controller : Endpoints pertaining to NIBRS Victim Demographic data (STATE)
#
#Modification History:
#   DD-MMM-YYY  Author          Description
#   10-07-2019  Stacey Smith    INITIAL CREATION
#   12-07-2019  Andrea Morgan   API loop build
#   15-07-2019  Stacey Smith    Made a copy of the original file to modify for economic analysis/breakdown age groups


In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import csv

from pprint import pprint

# Import DATA.GOV API key ......................REMEMBER TO PLUG IN YOUR OWN KEY NAME HERE
from api_keys import am_dg_key

In [9]:
#Clunky but works.  There are 3 pages of data for states.  
#This pulls the data for each page and combines it all into one usable dataframe: state_data

state_qry = "https://api.usa.gov/crime/fbi/sapi/api/states?api_key=" + am_dg_key 
state_results_1 = requests.get(state_qry, params={'page':0}).json()
state_results_2 = requests.get(state_qry, params={'page':1}).json()
state_results_3 = requests.get(state_qry, params={'page':2}).json()

sr1_df = pd.DataFrame(state_results_1['results'])
sr2_df = pd.DataFrame(state_results_2['results'])
sr3_df = pd.DataFrame(state_results_3['results'])


state_data = pd.concat([sr1_df, sr2_df, sr3_df])
state_data.head()

state_data.to_csv("output_data/state_data.csv")

In [10]:
#This is the same series created before
states = (state_data.iloc[:,1]).astype(str)

#Opened an empty list to initialize a dataframe
empty_list = []

ageDF = pd.DataFrame(empty_list)

#Creating columns on list based on the returned data for one state
ageDF["State"] = ""
ageDF["Year"] = ""
ageDF["Unknown"] = ""
ageDF["0-9"] = ""
ageDF["10-19"] = ""
ageDF["20-29"] = ""
ageDF["30-39"] = ""
ageDF["40-49"] = ""
ageDF["50-59"] = ""
ageDF["60-69"] = ""
ageDF["70-79"] = ""
ageDF["80-89"] = ""
ageDF["90-99"] = ""


#This is the loop to fill the dataframe.  This should work for any of us pulling the data from the FBI
#state APIs, with just tweaking the url, keyword and return information.

#The loop stores the return results for an individual state.  Then it loops through the return results to 
#populate each row in the empty dataframe created before.  Once it has looped through all of the return info,
#it goes back to the beginning and does the same for the next state in states.

#I found the "dataframe.append" bit on a website and it turned out to be the key to filling in the dataframe
#because it ignores index numbers and just appends to the next empty row available.  

for i in states:
    
    state = i 
    
    age_query_url = "https://api.usa.gov/crime/fbi/sapi/api/data/nibrs/homicide/victim/states/"+ state +"/age?api_key=" + am_dg_key
    age_response_state = requests.get(age_query_url).json()
    
    for i in age_response_state['results']:
        
        ageDF = ageDF.append({'State': state, 
                          'Year': i['data_year'], 
                          'Unknown': i['unknown'],
                          '0-9' : i['range_0_9'],
                          '10-19' : i['range_10_19'],
                          '20-29' : i['range_20_29'],
                          '30-39' : i['range_30_39'],
                          '40-49' : i['range_40_49'],
                          '50-59' : i['range_50_59'],
                          '60-69' : i['range_60_69'],
                          '70-79' : i['range_70_79'],
                          '80-89' : i['range_80_89'],
                          '90-99' : i['range_90_99']},
                          ignore_index = True)
    

ageDF  

,State,Year,Unknown,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90-99
0,AL,1991,28,11,54,131,105,49,16,20,12,7,1
1,AL,1992,12,10,33,67,70,43,16,15,7,5,0
2,AL,2006,0,0,0,2,0,0,0,0,0,0,0
3,AL,2008,0,0,0,0,0,1,1,0,0,0,0
4,AL,2009,0,0,0,1,1,0,0,0,0,0,0
5,AL,2010,0,0,0,1,0,0,0,0,0,0,0
6,AL,2012,0,0,1,0,0,0,0,1,0,0,0
7,AL,2013,0,0,0,0,2,0,0,0,0,0,0
8,AL,2014,0,0,0,1,0,0,0,0,0,0,0
9,AL,2015,0,0,0,1,1,1,0,0,0,0,0


In [12]:
ageDF.to_csv("output_data/ageDF.csv") 


#need to break down the 10-19 age range per age so I can calculate 16+ and 18+

In [16]:
#Opened an empty list to initialize a dataframe
empty_list = []

sexDF = pd.DataFrame(empty_list)

#Creating columns on list based on the returned data for one state
sexDF["State"] = ""
sexDF["Year"] = ""
sexDF["Female Count"] = ""
sexDF["Male Count"] = ""
sexDF["Unknown"] = ""


#This is the loop to fill the dataframe.  This should work for any of us pulling the data from the FBI
#state APIs, with just tweaking the url, keyword and return information.

#The loop stores the return results for an individual state.  Then it loops through the return results to 
#populate each row in the empty dataframe created before.  Once it has looped through all of the return info,
#it goes back to the beginning and does the same for the next state in states.

#I found the "dataframe.append" bit on a website and it turned out to be the key to filling in the dataframe
#because it ignores index numbers and just appends to the next empty row available.  

for i in states:
    
    state = i 
    
    sex_query_url = "https://api.usa.gov/crime/fbi/sapi/api/data/nibrs/homicide/victim/states/"+ state +"/sex?api_key=" + am_dg_key
    sex_response_state = requests.get(sex_query_url).json()
    
    for i in sex_response_state['results']:
        
        sexDF = sexDF.append({'State': state, 
                          'Year': i['data_year'], 
                          'Female Count': i['female_count'],
                          'Male Count' : i['male_count'],
                          'Unkown' : i['unknown']},
                          ignore_index = True)
    
    
sexDF.to_csv("output_data/sexDF.csv") 
sexDF  

,State,Year,Female Count,Male Count,Unknown,Unkown
0,AL,1991,107,321,NaN,6.0
1,AL,1992,54,216,NaN,8.0
2,AL,2006,1,1,NaN,0.0
3,AL,2008,1,1,NaN,0.0
4,AL,2009,1,1,NaN,0.0
5,AL,2010,0,1,NaN,0.0
6,AL,2012,0,2,NaN,0.0
7,AL,2013,2,0,NaN,0.0
8,AL,2014,1,0,NaN,0.0
9,AL,2015,1,2,NaN,0.0


In [17]:
#Opened an empty list to initialize a dataframe
empty_list = []

relDF = pd.DataFrame(empty_list)


#Creating columns on list based on the returned data for one state
relDF["State"] = ""
relDF["Year"] = ""
relDF["Relationship Unknown"] = ""
relDF["Acquaintance"] = ""
relDF["Babysittee"] = ""
relDF["BF_GF"] = ""
relDF["Child"] = ""
relDF["Child_BF_GF"] = ""
relDF["Common Law Spouse"] = ""
relDF["Employee"] = ""
relDF["Employer"] = ""
relDF["Ex Spouse"] = ""
relDF["Friend"] = ""
relDF["Grandchild"] = ""
relDF["Grandparent"] = ""
relDF["Same Sex Relationship"] = ""
relDF["In Law"] = ""
relDF["Neighbor"] = ""
relDF["Offender"] = ""
relDF["Other Family Member"] = ""
relDF["Otherwise Known"] = ""
relDF["Parent"] = ""
relDF["Sibling"] = ""
relDF["Spouse"] = ""
relDF["Stepchild"] = ""
relDF["Stepparent"] = ""
relDF["Stepsibling"] = ""
relDF["Stranger"] = ""


#This is the loop to fill the dataframe.  This should work for any of us pulling the data from the FBI
#state APIs, with just tweaking the url, keyword and return information.

#The loop stores the return results for an individual state.  Then it loops through the return results to 
#populate each row in the empty dataframe created before.  Once it has looped through all of the return info,
#it goes back to the beginning and does the same for the next state in states.

#I found the "dataframe.append" bit on a website and it turned out to be the key to filling in the dataframe
#because it ignores index numbers and just appends to the next empty row available.  

for i in states:
    
    state = i 
    
    rel_query_url = "https://api.usa.gov/crime/fbi/sapi/api/data/nibrs/homicide/victim/states/"+ state +"/relationship?api_key=" + am_dg_key
    rel_response_state = requests.get(rel_query_url).json()
    
    for i in rel_response_state['results']:
        
        relDF = relDF.append({'State': state, 
                          'Year': i['data_year'], 
                          'Relationship Unknown': i['relationship_unknown'],
                          'Acquaintance' : i['acquaintance'],
                          'Babysittee' : i['babysittee'],
                          'BF_GF' : i['boyfriend_girlfriend'],
                          'Child' : i['child'],
                          'Child_BF_GF' : i['child_boyfriend_girlfriend'],
                          'Common Law Spouse' : i['common_law_spouse'],
                          'Employee' : i['employee'],
                          'Employer' : i['employer'],
                          'Ex Spouse' : i['ex_spouse'],
                          'Friend' : i['friend'],
                          'Grandchild' : i['grandchild'],
                             'Grandparent' : i['grandparent'], 
                             'Same Sex Relationship' : i['homosexual_relationship'], 
                             'In Law' : i['in_law'], 
                             'Neighbor' : i['neighbor'], 
                             'Offender' : i['offender'], 
                             'Other Family Member' : i['other_family_member'], 
                             'Otherwise Known' : i['otherwise_known'], 
                             'Parent' : i['parent'], 
                             'Relationship Unkown' : i['relationship_unknown'], 
                             'Sibling' : i['sibling'],
                             'Spouse' : i['spouse'],
                             'Stepchild' : i['stepchild'],
                             'Stepparent' : i['stepparent'],
                             'Stepsibling' : i['stepsibling'],
                             'Stranger' : i['stranger']},
                          ignore_index = True)
    
    
relDF.to_csv("output_data/relDF.csv") 
relDF  

,State,Year,Relationship Unknown,Acquaintance,Babysittee,BF_GF,Child,Child_BF_GF,Common Law Spouse,Employee,...,Other Family Member,Otherwise Known,Parent,Sibling,Spouse,Stepchild,Stepparent,Stepsibling,Stranger,Relationship Unkown
0,AL,1991,86,136,0,48,9,0,4,3,...,16,1,7,6,38,0,2,0,179,86.0
1,AL,1992,126,76,0,17,7,0,5,0,...,11,1,3,4,20,1,1,0,45,126.0
2,AL,2006,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,0.0
3,AL,2008,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0.0
4,AL,2009,2,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.0
5,AL,2010,0,0,0,0,0,0,0,0,...,0,3,0,0,0,0,0,0,0,0.0
6,AL,2012,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0.0
7,AL,2013,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
8,AL,2014,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0.0
9,AL,2015,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0.0
